## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-29-10-52-00 +0000,wsj,"UnitedHealth Profit Shrinks, Reinstated Outloo...",https://www.wsj.com/business/earnings/unitedhe...
1,2025-07-29-10-52-00 +0000,wsj,"Stock Market Today: Dow, S&P 500 Futures Rise ...",https://www.wsj.com/livecoverage/stock-market-...
2,2025-07-29-10-51-30 +0000,nyt,Heavy Rain and Flooding in China Kill at Least 30,https://www.nytimes.com/2025/07/28/world/asia/...
3,2025-07-29-10-49-00 +0000,wsj,Trade Deal Brings Relief to Europe’s Luxury In...,https://www.wsj.com/business/retail/trade-deal...
4,2025-07-29-10-34-00 +0000,wsj,"Famine Is Unfolding in Gaza Strip, Experts Say",https://www.wsj.com/world/middle-east/experts-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
46,trump,71
22,trade,23
23,deal,20
96,new,19
135,shooting,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
250,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,147
192,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,144
28,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,142
221,2025-07-28-16-27-53 +0000,nypost,Trump hails cease-fire win after halting trade...,https://nypost.com/2025/07/28/us-news/trump-ha...,138
208,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,134


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
250,147,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
106,79,2025-07-29-00-38-47 +0000,nypost,NYC gunman ID’d as Shane Tamura after deadly s...,https://nypost.com/2025/07/28/us-news/nyc-gunm...
208,63,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
65,56,2025-07-29-06-07-59 +0000,bbc,Police officer among four dead in shooting at ...,https://www.bbc.com/news/articles/cn023751713o...
94,44,2025-07-29-02-05-13 +0000,nyt,Israel Returned to War in Gaza Hoping to Break...,https://www.nytimes.com/2025/07/28/world/israe...
247,40,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
50,37,2025-07-29-08-17-56 +0000,nypost,Sen. John Kennedy weighs in on Park Ave skyscr...,https://nypost.com/2025/07/29/us-news/sen-john...
258,34,2025-07-28-14-10-37 +0000,wapo,Thailand and Cambodia agree to ceasefire after...,https://www.washingtonpost.com/world/2025/07/2...
103,34,2025-07-29-01-00-00 +0000,wsj,Federal Reserve officials expect they will nee...,https://www.wsj.com/economy/central-banking/a-...
214,33,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
